In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    {"input": "안녕 내 이름은 김춘식이야."},
    {"output": "아니요 당신의 이름은 박춘식이기 때문에 박춘식으로 부르겠습니다."},
)

memory.load_memory_variables({})

{'history': 'Human: 안녕 내 이름은 김춘식이야.\nAI: 아니요 당신의 이름은 박춘식이기 때문에 박춘식으로 부르겠습니다.'}